#Cloning the GITHUB

In [ ]:
###### Set Up #####
# verify our folder with the data and module assets is installed
# if it is installed make sure it is the latest
!test -e ds-assets && cd ds-assets && git pull && cd ..
# if it is not installed clone it 
!test ! -e ds-assets && git clone https://github.com/IndraniMandal/ds-assets.git
# point to the folder with the assets
home = "ds-assets/assets/" 
import sys
sys.path.append(home)  

Cloning into 'ds-assets'...
remote: Enumerating objects: 176, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 176 (delta 3), reused 2 (delta 0), pack-reused 164
Receiving objects: 100% (176/176), 9.35 MiB | 12.60 MiB/s, done.
Resolving deltas: 100% (63/63), done.


#Obtaining Data

###The libaries/modules been implemented

In [ ]:
# setup
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from confint import classification_confint
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
from treeviz import tree_print

In [ ]:
print("******** data **********")

# get the newsgroup database
training_data_link = "https://raw.githubusercontent.com/lutzhamel/fake-news/master/data/fake_or_real_news.csv"
newsgroups = pd.read_csv(training_data_link)
#newsgroups = pd.read_csv(home+"newsgroups.csv")
newsgroups.head(n=10)

******** data **********


,id,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
5,6903,"Tehran, USA","\nI’m not an immigrant, but my grandparents ...",FAKE
6,7341,Girl Horrified At What She Watches Boyfriend D...,"Share This Baylee Luciani (left), Screenshot o...",FAKE
7,95,‘Britain’s Schindler’ Dies at 106,A Czech stockbroker who saved more than 650 Je...,REAL
8,4869,Fact check: Trump and Clinton at the 'commande...,Hillary Clinton and Donald Trump made some ina...,REAL
9,2909,Iran reportedly makes new push for uranium con...,Iranian negotiators reportedly have made a las...,REAL


In [ ]:
rows, cols = newsgroups.shape
print("The total number of rows is {} and the total number of columns is {}".format(rows,cols))

The total number of rows is 6335 and the total number of columns is 4


#Stops Words

Filtering the information that doesn't add concept to reduce the dimensionality of the feature space by removing words such as "the", "for", "that", etc.

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# get the newsgroup data
newsgroups = pd.read_csv(training_data_link)

# process documents                                                                                               
vectorizer = CountVectorizer(analyzer = "word", 
                             token_pattern = "[a-zA-Z]+", # only words
                             binary = True, 
                             stop_words = 'english',
                             min_df=2) # each word has to appear at least twice
docarray = vectorizer.fit_transform(newsgroups['text']).toarray()
                                                                                                 
print("docarray shape: {}".format(docarray.shape))
print("first 10 coords: {}".format(vectorizer.get_feature_names()[:10]))

docarray shape: (6335, 38240)
first 10 coords: ['aa', 'aaa', 'aab', 'aadmi', 'aaib', 'aam', 'aamaq', 'aap', 'aaron', 'aarp']


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
print("******** docarray **********")

# build the stemmer object
stemmer = PorterStemmer()

# build a new default analyzer using CountVectorizer that only uses words: [a-zA-Z]+
# also eliminate stop words
analyzer= CountVectorizer(analyzer = "word", 
                          stop_words = 'english',
                          token_pattern = "[a-zA-Z]+").build_analyzer()

# build a new analyzer that stems using the default analyzer to create the words to be stemmed
def stemmed_words(doc):
    return [stemmer.stem(w) for w in analyzer(doc)]

# build docarray
vectorizer = CountVectorizer(analyzer=stemmed_words,
                             #analyzer=analyzer,
                             binary=True,
                             min_df=2) # each word has to appear at least twice
docarray = vectorizer.fit_transform(newsgroups['text']).toarray()
docarray.shape
doc_df = pd.DataFrame(docarray, columns=list(vectorizer.get_feature_names()))
doc_df.head()

******** docarray **********


,aa,aaa,aab,aadmi,aaib,aam,aamaq,aap,aaron,aarp,...,zuckerberg,zuckerburg,zuckerman,zuess,zukowski,zulu,zurich,zvezda,zvz,zwick
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:

doc_names = ['doc{}'.format(i) for i in range(newsgroups.shape[0])]
newsgroups = pd.DataFrame(newsgroups.values, index=doc_names,columns=newsgroups.columns)
print(newsgroups.head(n=10))

# build the stemmer object
stemmer = PorterStemmer()

# get the default text analyzer from CountVectorizer
analyzer = CountVectorizer(analyzer = "word", 
                           stop_words = 'english',
                           token_pattern = "[a-zA-Z]+").build_analyzer()

# build a new analyzer that stems using the default analyzer to create the words to be stemmed
def stemmed_words(doc):
    return [stemmer.stem(w) for w in analyzer(doc)]

vectorizer = CountVectorizer(analyzer=stemmed_words,
                                 binary=True,
                                 min_df=2)
docarray = vectorizer.fit_transform(newsgroups['text']).toarray()

print("docarray shape: {}".format(docarray.shape))
print("first 10 coords: {}".format(vectorizer.get_feature_names()[:10]))

         id                                              title  \
doc0   8476                       You Can Smell Hillary’s Fear   
doc1  10294  Watch The Exact Moment Paul Ryan Committed Pol...   
doc2   3608        Kerry to go to Paris in gesture of sympathy   
doc3  10142  Bernie supporters on Twitter erupt in anger ag...   
doc4    875   The Battle of New York: Why This Primary Matters   
doc5   6903                                        Tehran, USA   
doc6   7341  Girl Horrified At What She Watches Boyfriend D...   
doc7     95                  ‘Britain’s Schindler’ Dies at 106   
doc8   4869  Fact check: Trump and Clinton at the 'commande...   
doc9   2909  Iran reportedly makes new push for uranium con...   

                                                   text label  
doc0  Daniel Greenfield, a Shillman Journalism Fello...  FAKE  
doc1  Google Pinterest Digg Linkedin Reddit Stumbleu...  FAKE  
doc2  U.S. Secretary of State John F. Kerry said Mon...  REAL  
doc3  — Kaydee Ki

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


#Decision Tree

In [ ]:
print("******** model **********")


# Decision Tree
model = DecisionTreeClassifier(random_state=0)

# grid search
param_grid = {'max_depth': list(range(1,31))}
grid = GridSearchCV(model, param_grid, cv=3, verbose=10, n_jobs=-1)
grid.fit(docarray, newsgroups['label'])
print("Grid Search: best parameters: {}".format(grid.best_params_))
tree_print(grid.best_estimator_,doc_df)

******** model **********
Fitting 3 folds for each of 30 candidates, totalling 90 fits


/usr/local/lib/python3.8/dist-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Grid Search: best parameters: {'max_depth': 30}
if republican =< 0.5: 
  |then if octob =< 0.5: 
  |  |then if obama =< 0.5: 
  |  |  |then if said =< 0.5: 
  |  |  |  |then if verdict =< 0.5: 
  |  |  |  |  |then if sen =< 0.5: 
  |  |  |  |  |  |then if buzhardt =< 0.5: 
  |  |  |  |  |  |  |then if dem =< 0.5: 
  |  |  |  |  |  |  |  |then if washburn =< 0.5: 
  |  |  |  |  |  |  |  |  |then if candid =< 0.5: 
  |  |  |  |  |  |  |  |  |  |then if incom =< 0.5: 
  |  |  |  |  |  |  |  |  |  |  |then if conserv =< 0.5: 
  |  |  |  |  |  |  |  |  |  |  |  |then if tariff =< 0.5: 
  |  |  |  |  |  |  |  |  |  |  |  |  |then if com =< 0.5: 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |then if ohio =< 0.5: 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |then if muhammad =< 0.5: 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |then if tammi =< 0.5: 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |then if capitol =< 0.5: 
  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |then if charlest

In [ ]:
print("******** Accuracy **********")

# accuracy of best model with confidence interval
best_model = grid.best_estimator_
predict_y = best_model.predict(docarray)
acc = accuracy_score(newsgroups['label'], predict_y)
lb,ub = classification_confint(acc,docarray.shape[0])
print("Accuracy: {:3.2f} ({:3.2f},{:3.2f})".format(acc,lb,ub))

******** Accuracy **********
Accuracy: 0.98 (0.98,0.99)


##KNN

In [ ]:
print("******** model **********")


# KNN
model = KNeighborsClassifier()

# grid search
param_grid = {'n_neighbors': list(range(1,11))}
grid = GridSearchCV(model, param_grid, cv=3, verbose=10, n_jobs=-1)
grid.fit(docarray, newsgroups['label'])
print("Grid Search: best parameters: {}".format(grid.best_params_))

******** model **********
Fitting 3 folds for each of 10 candidates, totalling 30 fits


/usr/local/lib/python3.8/dist-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Grid Search: best parameters: {'n_neighbors': 1}


In [ ]:
print("******** Accuracy **********")

# accuracy of best model with confidence interval
best_model = grid.best_estimator_
predict_y = best_model.predict(docarray)
acc = accuracy_score(newsgroups['label'], predict_y)
lb,ub = classification_confint(acc,docarray.shape[0])
print("Accuracy: {:3.2f} ({:3.2f},{:3.2f})".format(acc,lb,ub))

******** Accuracy **********
Accuracy: 1.00 (1.00,1.00)


In [ ]:
print("******** confusion matrix **********")

# build the confusion matrix
cats = ['FAKE','REAL']
cm = confusion_matrix(newsgroups['label'], predict_y, labels=cats)
cm_df = pd.DataFrame(cm, index=cats, columns=cats)
print("Confusion Matrix:\n{}".format(cm_df))

******** confusion matrix **********
Confusion Matrix:
      FAKE  REAL
FAKE  3164     0
REAL     0  3171


## Naive Bayes (NB)

In [ ]:

from sklearn.naive_bayes import MultinomialNB 
## Naive Bayes

print("******** model **********")


# Naive Bayes
model = MultinomialNB()
# NOTE: NB does not have any hyper-parameters - no overfitting - no searching over parameter space!
model.fit(docarray, newsgroups['label'])


print("******** Accuracy **********")

# accuracy of best model with confidence interval
best_model = model
predict_y = best_model.predict(docarray)
acc = accuracy_score(newsgroups['label'], predict_y)
lb,ub = classification_confint(acc,docarray.shape[0])
print("Accuracy: {:3.2f} ({:3.2f},{:3.2f})".format(acc,lb,ub))

print("******** confusion matrix **********")

# build the confusion matrix
cats = ['FAKE','REAL']
cm = confusion_matrix(newsgroups['label'], predict_y, labels=cats)
cm_df = pd.DataFrame(cm, index=cats, columns=cats)
print("Confusion Matrix:\n{}".format(cm_df))

******** model **********
******** Accuracy **********
Accuracy: 0.92 (0.91,0.93)
******** confusion matrix **********
Confusion Matrix:
      FAKE  REAL
FAKE  2856   308
REAL   197  2974


###The preprocessed models had better accuracy than the processed model